<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2021notebooks/2021_1003vanilla_seq2seq2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
# Google Colaboratory では GPU モードで実行してください

- date: 2021_1003
- author: 浅川伸一
- licese: MIT
- filename: 2021_1003vanilla_seq2seq.ipynb

# バニラ seq2seq



In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import random

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
import sys
import requests
url_base = 'https://komazawa-deep-learning.github.io/2021/'
jpntxt = 'eng-jpn.txt'
chntxt = 'eng-chn.txt'
for _lang in [jpntxt,chntxt]:
    r = requests.get(url_base+_lang, _lang)
    with open('___'+_lang, 'wb') as f:
        f.write(r.content)

!mkdir data        
!wget https://komazawa-deep-learning.github.io/2021/lang_dict.py -O data/lang_dict.py
!wget https://komazawa-deep-learning.github.io/2021/eng-jpn_normalized.txt -O data/eng-jpn_normalized.txt
!wget https://komazawa-deep-learning.github.io/2021/eng-chn_normalized.txt -O data/eng-chn_normalized.txt

%load_ext autoreload
%autoreload 2
import data.lang_dict

C2E_dict = data.lang_dict.C2E_dict
E2C_dict = data.lang_dict.E2C_dict
J2E_dict = data.lang_dict.J2E_dict
E2J_dict = data.lang_dict.E2J_dict
jpn = data.lang_dict.jpn
chn = data.lang_dict.chn
E2J_pairs = data.lang_dict.E2J_pairs
E2C_pairs = data.lang_dict.E2C_pairs
print(E2J_pairs[:5])

MAX_LENGTH = data.lang_dict.MAX_LENGTH
teacher_forcing_ratio = 0.5
SOS_token = 1
EOS_token = 0

--2021-10-07 23:53:53--  https://komazawa-deep-learning.github.io/2021/lang_dict.py
Resolving komazawa-deep-learning.github.io (komazawa-deep-learning.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to komazawa-deep-learning.github.io (komazawa-deep-learning.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2218 (2.2K) [application/octet-stream]
Saving to: ‘data/lang_dict.py’

data/lang_dict.py   100%[===================>]   2.17K  --.-KB/s    in 0s      

2021-10-07 23:53:53 (26.0 MB/s) - ‘data/lang_dict.py’ saved [2218/2218]

--2021-10-07 23:53:53--  https://komazawa-deep-learning.github.io/2021/eng-jpn_normalized.txt
Resolving komazawa-deep-learning.github.io (komazawa-deep-learning.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to komazawa-deep-learning.github.io (komazawa-deep-learning.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response

In [3]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%02dm %02ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '経過時間:%s (残り時間:%s)' % (asMinutes(s), asMinutes(rs))

In [4]:
import torch.nn as nn  # 冗長に import するのは教育的配慮からである

class EncoderRNN(nn.Module):
    def __init__(self, n_inp, n_hid, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.n_hid = n_hid
        self.embed = nn.Embedding(n_inp, n_hid)
        self.rnn = nn.LSTM(n_hid, n_hid)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, inp, hid):
        embed = self.embed(inp).view(1, 1, -1)
        out = self.dropout(embed)
        out, (C, hid) = self.rnn(out, hid)
        out = self.dropout(out)
        return out, (C, hid)

    def initHidden(self):
        weight = next(self.parameters())
        return (weight.new_zeros(1, 1, self.n_hid),
                weight.new_zeros(1, 1, self.n_hid))

    

class DecoderRNN(nn.Module):
    def __init__(self, n_hid, n_out, dropout_p=0.1):
        super(DecoderRNN, self).__init__()
        self.n_hid = n_hid

        self.embed = nn.Embedding(n_out, n_hid)
        self.rnn = nn.LSTM(n_hid, n_hid)
        self.out = nn.Linear(n_hid, n_out)
        self.softmax = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input, hidden):
        output = self.embed(input).view(1, 1, -1)
        output = self.dropout(output)
        output = F.relu(output)
        output, (C, hidden) = self.rnn(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, (C, hidden)

    def initHidden(self):
        weight = next(self.parameters())
        return (weight.new_zeros(1, 1, self.n_hid),
                weight.new_zeros(1, 1, self.n_hid))
        

# 2. 訓練

## 2.1 訓練データの作成

モデルを訓練するには、(英語, 他の言語)という言語対ごとに、入力データのテンソル（入力文に含まれる単語のインデックス）と正解データのテンソル（正解文に含まれる単語のインデックス）が必要となります。
入力データと正解データのテンソルを作成する処理のなかで、EOS（End of Sentence）トークンを入力データ， 正解データの各シーケンスに追加する処理も行います。


In [5]:
def indexesFromSentence(lang_dict, sentence):
    return [lang_dict.wrd2idx[word] for word in sentence.split(' ')]


def tensorFromSentence(lang_dict, sentence):
    indexes = indexesFromSentence(lang_dict, sentence)
    indexes.append(lang_dict.wrd2idx['EOS'])
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair, SRC, DST):
    input_tensor = tensorFromSentence(SRC, pair[0])
    target_tensor = tensorFromSentence(DST, pair[1])
    return (input_tensor, target_tensor)


## 2.2 モデルの訓練

* 訓練するには、入力文を符号化器に投入して実行し、実行の過程のすべての出力と、一番最後の隠れ状態を記録します。
* 続いて、復号化器には最初の入力として トークンが渡され、また、エンコーダの最後の隠れ状態が、デコーダの最初の隠れ状態として渡されます。
* ここで、教師強制（Teacher forcing）と呼ばれるアイデアを紹介します。
* 「教師強制」は 復号化器の推論結果を次のステップの RNN への入力として使用するのではなく、次のステップでは、実際の正解データを RNN への入力として使用するという手法です。
* 教師強制を使用すると、より速く収束しますが、訓練されたネットワークが悪用されると不安定になり問題になることがあります.
* 教師強制のネットワークの出力は、文法的には正しくても、正しい翻訳からはかけ離れたものになることがあります。
* 教師強制の直感的な理解としては、以下のようなものとなります。
* まず、教師強制は「文法」を表現することを学習させています。
* また、正しい翻訳の最初の数語を伝えればそこから意味を「拾う」ことはできます。
* しかし、そもそも翻訳で文章を作る方法をきちんと学べてはいません。
* PyTorch の `autograd` の自由度のおかげで、単純な if 文だけで、教師強制を使うか使わないかをランダムに選択することができます。
* `teacher_forcing_ratio` の値を上げると、より頻繁に教師強制を使うようになります。

In [6]:
MAX_LENGTH = data.lang_dict.MAX_LENGTH
teacher_forcing_ratio = 0.5
SOS_token = 1
EOS_token = 0

In [7]:
def vanilla_train(input_tensor, target_tensor, 
                  encoder, decoder, 
                  encoder_optimizer, decoder_optimizer, 
                  criterion, max_length=MAX_LENGTH,
                  teacher_forcing_ratio = 0.5
                 ):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.n_hid, device=device)

    loss = 0
    
    # 符号化器側の処理
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

   
    decoder_input = torch.tensor([[SOS_token]], device=device)
    
    # 符号化器の中間層の状態を復号化器の中間層状態に接続する
    decoder_hidden = encoder_hidden
    
    # 復号化器側の処理
    # 教師強制をするか，否か，の判断
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # 教師強制の場合：次の入力として正解データを渡す。
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
         # 教師強制を使わない：自分の予測を次の入力として使用する
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 入力として使用するために、計算グラフから切り離す

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import matplotlib.pyplot as plt

# 目盛りの調整のたｍ matplotlib.ticker を使用
import matplotlib.ticker as ticker

! pip install japanize_matplotlib
import japanize_matplotlib
import numpy as np

def draw_curve(points, figsize=(5,5)):
    plt.figure(figsize=figsize)
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.25) # 等間隔で tic を設定
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)


In [10]:
def trainIters(encoder, decoder, 
               SRC, DST,
               pairs=E2J_pairs,
               train_f=vanilla_train, 
               n_iters=75000, 
               print_interval=1000, plot_interval=100, 
               learning_ratio=0.01):
    """
    引数:
    encoder: nn.Module.RNNmodel
        符号化器。ソース言語
    decoder: nn.Module.RNNmodel
        復号化器。ターゲット言語
    n_iters: int
        反復回数
    
    注: ミニバッチを用いずにオンライン学習によって学習を行う。
    """
    
    start = time.time()  # 開始時刻を保存しておく

    # 損失関数の値を初期化
    plot_losses = []
    print_loss_total = 0  # printする度にリセット
    plot_loss_total = 0   # plot_everyごとにリセット

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_ratio)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_ratio)
    
    # 乱数を用いて訓練データを選択。ここは perminatation した方が良さそうだな
    # 加えてこれは，エポックにしてないな
    training_pairs = [tensorsFromPair(random.choice(pairs), SRC, DST)
                      for i in range(n_iters)]
    
    criterion = nn.NLLLoss()  # 負の対数尤度を学習基準とする

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train_f(input_tensor, target_tensor, encoder,
                       decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_interval == 0:
            print_loss_avg = print_loss_total / print_interval
            print_loss_total = 0
            print(f'平均損失: {print_loss_avg:.3f}',
                  f'(反復回数: {iter:5d} {iter/n_iters * 100:5.2f}%)',
                  f'{timeSince(start, iter / n_iters)}'
                 )

        if iter % plot_interval == 0:
            plot_loss_avg = plot_loss_total / plot_interval
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    draw_curve(plot_losses)
    return plot_losses


In [ ]:
n_hid = 64

encoder = EncoderRNN(E2J_dict.n_words, n_hid).to(device)
decoder = DecoderRNN(n_hid, J2E_dict.n_words, dropout_p = 0.1).to(device)
_ = trainIters(encoder, decoder, E2J_dict, J2E_dict, pairs=E2J_pairs, 
               n_iters=2000, print_interval=500, learning_ratio=0.001)

In [ ]:
_ = trainIters(encoder, decoder, E2J_dict, J2E_dict, pairs=E2J_pairs, n_iters=20000, print_interval=5000, learning_ratio=0.0001)

In [ ]:
# 読み出し
cptfile = '2021_1003valina_seq2seq2.cpt'
cpt = torch.load(cptfile)
encoder_sd = cpt['encoder_state_dict']
decoder_sd = cpt['decoder_state_dict']
encoder.load_state_dict(encoder_sd)
decoder.load_state_dict(decoder_sd)

<All keys matched successfully>

In [ ]:
trainIters(encoder, decoder, E2J_dict, J2E_dict, pairs=E2J_pairs, n_iters=1000, print_interval=250)

In [ ]:
losses = trainIters(encoder, decoder, E2J_dict, J2E_dict, pairs=E2J_pairs, n_iters=1000, print_interval=100, plot_interval=50)

# 3. 評価

評価はモデルの訓練とほとんど同じですが、正解データがないので、各ステップごとにデコーダの予測値を自分自身にフィードバックします。
復号化器が単語を予測するたびに， それを出力文字列に追加します。
そして予測結果が EOS トークンとなった場合には， そこで予測を停止します。
さらに、表示用に復号化器の注意を保存します

In [ ]:
def evaluate(encoder, decoder, SRC, DST, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(SRC, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.n_hid, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(DST.idx2wrd[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, SRC, DST, n=10):
    outs = []
    for i in range(n):
        pair = random.choice(E2J_pairs)
        print(f'ソース< {pair[0]}')
        print(f':正解= { pair[1]}')
        output_words = evaluate(encoder, decoder, SRC, DST, pair[0])
        output_sentence = ' '.join(output_words)
        print(f':翻訳> {output_sentence}\n')
        outs.append((pair[0],output_sentence))
    return outs

In [ ]:
_ = evaluateRandomly(encoder, decoder, E2J_dict, J2E_dict, n=3)

In [ ]:
cpt_fname = '2021_1008valina_seq2seq.cpt'
torch.save({'encoder_state_dict': encoder.state_dict(),
            'decoder_state_dict': decoder.state_dict(),
            #'loss': losses,
            'E2J_dict': E2J_dict, 
            'J2E_dict': J2E_dict, 
            'pairs': E2J_pairs,
           }, cpt_fname)